In [5]:
import pandas as pd
import re
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

service=Service('C:/Users/Manych/chromium/chromedriver.exe')

In [8]:
def get_marks_models():
    '''
    возврашщает pd.Series в котором 
    индекс - название марки автомобиоя, 
    значения - списки названий моделей для каждой маркию
    
    '''
    url_for_marks_models = 'https://auto.ru/catalog/cars/all/'
    marks_models = pd.Series() 

    driver = webdriver.Chrome(service=service)  # открываем driver
    driver.maximize_window()                    # масксимизируем окно

    for pages_num in range(1,20):
        
        if pages_num==1: url = url_for_marks_models + '?view_type=list'
        else:            url = url_for_marks_models + '?page_num='+ str(pages_num) +'&view_type=list'
        
        driver.get(url)                         # открываем страницу по url
        res = driver.execute_script("return document.body.innerHTML;") # получаем html
        soup = BeautifulSoup(res, 'html.parser') # создаем обьект bs4.BeautifulSoup
        
        # Список html-ек перечня моделей каждой марки на странице:
        marks_on_page_list = soup.find_all('dd', class_='catalog-all-text-list__desc') 
    
        if not marks_on_page_list: 
            break
        
        for mark in marks_on_page_list:
            models_of_mark = mark.find_all('a', class_='link_theme_auto') # список html-ек моделей марки
        
            link_for_mark_name = models_of_mark[0].get('href')  # линк первой модели, содержит обозначене марки на auto.ru
            mark_start = link_for_mark_name.find('cars/') + 5   # первый символ обозначеня марки
            mark_end = link_for_mark_name.find('/', mark_start) # последний символ обозначения марки
            mark_name = link_for_mark_name[mark_start:mark_end].upper() # получение обозначения марки  
        
            models_list = []
            for model in models_of_mark:
                link_theme_auto = model.get('href')                 # линк модели, содержит обозначение модели на auto.ru
                model_start = mark_end+1                            # первый символ обозначения модели
                model_end = link_theme_auto.find('/', model_start)  # последний символ обозначения модели
                model_name = link_theme_auto[model_start:model_end].upper() # получение обозначения модели
                models_list.append(model_name)
                
            marks_models[mark_name] = models_list
        
        time.sleep(1) 
    # Закрываем процесс браузера:
    driver.quit()
    
    return marks_models

def get_generation_year(model_url):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название модели с указанием поколения, 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    
    '''
    driver.get(model_url+'?output_type=models_list') # открываем страницу по url
    
    models_list_res = driver.execute_script("return document.body.innerHTML;") # получаем html        
    
    # создаем обьект bs4.BeautifulSoup из html: 
    models_list_bs = BeautifulSoup(models_list_res, 'html.parser') 
    
    # получаем список html-ек описаний для всех поколений модели  
    posting_tag = models_list_bs.find_all('h3',class_ = "ListingItemGroup__title")
    
    # формируем списки с указанием поколения модели, типом кузова поколения и годом начала выпуска поколения
    # (необходимость указания типа кузова вызвана тем, что разные кузова зачастую переходят на следующее 
    #  поколение в разные годы)
    data = [
    [tag.find('a',class_ = 'ListingItemTitle__link').text,             # поколение
     tag.find('div',class_ = "ListingItemGroup__subtitle").text[:      # кузов
                tag.find('div',class_ = "ListingItemGroup__subtitle").text.find(' •')],
     int(tag.find('div',class_ = "ListingItemGroup__subtitle").text    # год
                [tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+1:
                 tag.find('div',class_ = "ListingItemGroup__subtitle").text.find('(')+5])] 
        for tag in posting_tag]
    
    result = pd.DataFrame(data,columns = ['full_name','bodytype','generation_year'])
    
    return result 

def get_model_generation_year(marks_models_for_parsing):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название марки и модели с указанием поколения 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
    
    marks_models_for_parsing - pd.Series в котором
        индекс: марка автомобиля
        значение: список моделей данной марки
    '''
    driver = webdriver.Chrome(service=service)  # запускаем процесс браузера
    driver.maximize_window()                    # масксимизируем окно    
    
    model_generation_year = pd.DataFrame(columns = ['full_name','bodytype','generation_year'])
    
    for mark in marks_models_for_parsing.index:
        print(mark, end=' | ')
        for model in marks_models_for_parsing[mark]:
            model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
            tmp = get_generation_year(model_url)
            model_generation_year = model_generation_year.append(tmp)
            time.sleep(1)
            
    driver.quit()    # закрываем процесс браузера
    
    return model_generation_year

In [9]:
%%time
marks_models = get_marks_models()

<ipython-input-8-1f609f36d03a>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  marks_models = pd.Series()


Wall time: 1min


In [10]:
marks_for_parsing = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
       'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
marks_models_for_parsing = marks_models.loc[marks_for_parsing]

In [10]:
%%time
model_generation_year_all =  get_model_generation_year(marks_models_for_parsing)

SKODA | AUDI | HONDA | VOLVO | BMW | NISSAN | INFINITI | MERCEDES | TOYOTA | LEXUS | VOLKSWAGEN | MITSUBISHI | Wall time: 1h 23min 54s


In [15]:
# model_generation_year_all.to_csv('./Project_6_data/model_generation_year_all.csv',index=False)
model_generation_year_all = pd.read_csv('./Project_6_data/model_generation_year_all.csv')

In [16]:
model_generation_year_all.head(15)

,full_name,bodytype,generation_year
0,Skoda Fabia II,хэтчбек 5 дв.,2007
1,Skoda Fabia II Рестайлинг,хэтчбек 5 дв.,2010
2,Skoda Fabia II,универсал 5 дв.,2007
3,Skoda Fabia I Рестайлинг,хэтчбек 5 дв.,2005
4,Skoda Fabia I,хэтчбек 5 дв.,2001
5,Skoda Fabia I,седан,2002
6,Skoda Fabia I,универсал 5 дв.,2000
7,Skoda Fabia II Рестайлинг,универсал 5 дв.,2010
8,Skoda Fabia I Рестайлинг,универсал 5 дв.,2005
9,Skoda Fabia RS II,универсал 5 дв.,2010


In [12]:
marks_models_for_parsing.SKODA         

['FABIA',
 'FABIA_RS',
 'FELICIA',
 'FORMAN',
 'KAROQ',
 'KODIAQ',
 'OCTAVIA',
 'OCTAVIA_RS',
 'RAPID',
 'ROOMSTER',
 'SUPERB',
 'YETI']

# ==============================================

def get_generation_year(model_url):
    '''
    возврашщает pd.DataFrame  в котором: 
        full_name - полное название модели с указанием поколения, 
        bodytype - тип кузова поколения модели
        generation_year - год начала выпуска поколения
        
    model_url - ссылка на страницу со списокм поколений модели
    
    '''        
    # получем html 
    models_list_res = requests.get(model_url+'?output_type=models_list')
    models_list_res.encoding = 'utf-8'

    # создаем обьект bs4.BeautifulSoup из html 
    models_list_bs = BeautifulSoup(models_list_res.text, 'html.parser') 
    
    # получаем список html-ек описаний для всех поколений модели  
    posting_tag = models_list_bs.find_all('div',class_ = "ListingItemGroup")
    
    # формируем списки с указанием поколения модели, типом кузова поколения и годом начала выпуска поколения
    # (необходимость указания типа кузова вызвана тем, что разные кузова зачастую переходят на следующее 
    #  поколение в разные годы)
    data = [
    [tag.find('a',class_ = 'ListingItemTitle-module__link').text,             # поколение
     tag.find('div',class_ = "ListingItemTitle-module__subtitle").text[:      # кузов
                tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find(' •')],
     int(tag.find('div',class_ = "ListingItemTitle-module__subtitle").text    # год
                [tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find('(')+1:
                 tag.find('div',class_ = "ListingItemTitle-module__subtitle").text.find('(')+5])] 
        for tag in posting_tag]
    
    result = pd.DataFrame(data,columns = ['full_name','bodytype','generation_year'])

    return result

In [ ]:
# mark = 'BMW'
# model = '5ER'
# model_url = 'https://auto.ru/moskva/cars/' + mark.lower() + '/' + model.lower() + '/used/'
# model_url+'?output_type=models_list'

In [ ]:
# marks_models = get_marks_models()

In [ ]:
def getHumanName(url: str) -> str:
    try:
        soup = BeautifulSoup(
            requests.get(url).content.decode('utf-8'),'html.parser'
        )
        name = list(
            soup.find('div', class_='CardBreadcrumbs').children
        )
        name = name[-2].get_text()
    except:
        name = np.nan
    time.sleep(0.1)
    return name

# ========== PARALLEL ==============

In [ ]:
n_rec = [300, 500, 750, 1000]
n_jobs = [10, 15, 20, 25, 30, 35, 40]
dur = {}
for rec in n_rec:
    urls = data_test.car_url.sample(rec).tolist()
    delta = []
    for num in n_jobs:
        start = time.time()
        name_lst = Parallel(n_jobs=num)(delayed(getHumanName)(url) for url in urls)
        fin = time.time()
        delta.append(round(fin - start, 0))
    dur[rec] = delta

In [2]:
from math import sqrt
from joblib import Parallel, delayed

In [18]:
%%time
[sqrt(i ** 2) for i in range(10)];


Wall time: 0 ns


[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [ ]:
[[i,sqrt(i ** 2),sqrt(i ** 3)] for i in range(10)];

In [3]:
%%time
def a(x):
    return [x,x**2,x**3]
aa = Parallel(n_jobs=2)(delayed(a)(x) for x in range(10))
aa

Wall time: 473 ms


[[0, 0, 0],
 [1, 1, 1],
 [2, 4, 8],
 [3, 9, 27],
 [4, 16, 64],
 [5, 25, 125],
 [6, 36, 216],
 [7, 49, 343],
 [8, 64, 512],
 [9, 81, 729]]

In [13]:
import concurrent.futures